Below are the pre-requisite installs for the file, as the data was imported by the json API token method for accessibility.

In [16]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from keras.models import Model
from glob import glob

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import (
    Dense, Conv2D, MaxPool2D, Dropout, Flatten, 
    BatchNormalization,GlobalAveragePooling2D
)
from keras.layers import AveragePooling2D, MaxPooling2D

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16

from keras import backend as K

from sklearn.metrics import confusion_matrix, classification_report

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [19]:
!kaggle datasets download -d prashant268/chest-xray-covid19-pneumonia

100% 2.05G/2.06G [00:25<00:00, 114MB/s]
100% 2.06G/2.06G [00:25<00:00, 86.5MB/s]


In [20]:
!kaggle datasets download -d prashant268/chest-xray-covid19-pneumonia --unzip --force

 99% 2.04G/2.06G [00:25<00:00, 90.2MB/s]
100% 2.06G/2.06G [00:25<00:00, 85.6MB/s]


In terms of data cleaning/preprocessing:

From the assignment brief we are already informed that all images are already the same size, as such no adjustments were required in this case.

To counteract the issue of images being in a different rotation, we employed data augmentation in order to create copies of the data in different rotations, in order to get our models "used to" images in different rotations and other issues, so that greater accuracy can be achieved in the long-run.

In [21]:
train_Path = '/content/Data/train'
test_Path = '/content/Data/test'

Next, the data is augmented using Keras' ImageDataGenerator:


In [22]:
train_datagen = ImageDataGenerator(rotation_range=90,
                              fill_mode= 'nearest',
                              height_shift_range=0.3,
                              width_shift_range=0.3,
                              horizontal_flip= False,
                              vertical_flip= False,
                              brightness_range=[0.5,1.5],
                              zoom_range=0.2,
                              rescale=1./225,
                              samplewise_std_normalization=True,
                                   )


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [23]:
test_datagen2 = ImageDataGenerator(rotation_range=90,
                              fill_mode= 'nearest',
                              height_shift_range=0.3,
                              width_shift_range=0.3,
                              horizontal_flip= False,
                              vertical_flip= False,
                              brightness_range=[0.5,1.5],
                              zoom_range=0.2,
                              rescale=1./225,
                              featurewise_std_normalization= True,
                              samplewise_std_normalization= True
                                   )

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Creating ResNet and VGG net


In [24]:
baseModel1 = VGG16(input_shape=(224,224,3), weights='imagenet',include_top=False)
baseModel2 = ResNet50(input_shape=(224,224,3),weights='imagenet',include_top=False)

for layer in baseModel1.layers:
  layer.trainable = False

for layer in baseModel2.layers:
  layer.trainable = False


classes = glob('/content/Data/train/*')

vgg_last_layer = Flatten()(baseModel1.output)
resnet_last_layer = Flatten()(baseModel2.output)

vgg_prediction = Dense(len(classes), activation='softmax')(vgg_last_layer)
resnet_prediction = Dense(len(classes), activation='softmax')(resnet_last_layer)

VGG = Model(inputs = baseModel1.input, outputs= vgg_prediction)
ResNet = Model(inputs = baseModel2.input, outputs= resnet_prediction)



Next, by examining the summary of the models after altering the final layer, we can observe if the final layer has an output of 3 instead of 1000, which is the base version of this model:

In [25]:
VGG.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
ResNet.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

Below is the compiling of the model, adding in:

Optimiser = Adam was selected 

Loss function = Categorical crossentropy - 

Metrics = Examining the overall accuracy for part 1 was our only concern, as this was an exercise of simply examining how well the imagenet weights and base variant of the models worked with our dataset (with the minor alteration of 3 classes down from 1000)

In [26]:
from tensorflow.keras.optimizers import Adam


opt=Adam(learning_rate=0.0001)

VGG.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
ResNet.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])





training_set = train_datagen.flow_from_directory(train_Path,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 seed=42,
                                                 shuffle=True)



# This below version is just for the case in which we clean the test data beforehand:
test_set2 = test_datagen2.flow_from_directory(test_Path,
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            seed= 42,
                                            shuffle = False)


Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


Step 1 of assignment - Testing base variants of VGG and ResNet on our models, with the only adjustment being the output layer size being converted from 1000 to 3 (in respect to the our class count).

The results largely match our expectations, as the weights of these models were trained for a differing task and as such would perform poorly on our chest x-ray dataset as it has not been fitted to this task.



In [27]:
VGG16_test =  VGG.evaluate(test_set2) 

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 8/41 [====>.........................] - ETA: 10:02 - loss: 0.9732 - accuracy: 0.4961

KeyboardInterrupt: ignored

In [ ]:
Resnet_Test = ResNet.evaluate(test_set2)

As observed, the accuracy scores reported were:

VGG16: 25%
ResNet50: 54%

ResNet outperformed VGG16 considerably, and as observed above executed much faster than VGG16 (230s in comparison to 674s).

Whilst this stage isn't particularly defining for how the models will ultimately perform on the dataset after fitting/training, it is interesting to note that ResNet50 performed nearly 5x better than VGG16, and with a faster runtime.

VGG16's 12% was lacklustre but somewhat expected given the freezing of the imagenet weights which where of course trained for a different task.

ResNet50's 54% however was considerably better than expected given the circumstances and limitations placed on the model when applying it to the dataset, this could imply ResNet will be the de facto greater option after training/fitting also.




In [1]:
VGG16_test =  VGG.evaluate(test_set2) 

NameError: ignored

In [ ]:
Resnet_Test = ResNet.evaluate(test_set2)

Above shows the results of the base evaluation when the data was augmented/cleaned on the test set also, giving us slightly different scores of:

VGG16: 24%
ResNet50: 46%

The execution times were largely similar to the un-augmented variant of the test data as expected, with ResNet still being considerably faster to run in comparison to VGG16.

ResNet also performed better than VGG once again, with nearly twice the accuracy of the VGG16 model.


For part 2 of the assignment, we opted to modify our chosen network archetypes (VGG16 & ResNet50) in hopes to improve them:

Below is a modification of our ResNet model:


Whilst we may have created a sequential model and simply added the layers from the VGG16 network to the sequential model, with further modifications being made, we were unable to mimic this technique for ResNet as it was incompatible with the Sequential model call.

As we were unable to find a fix, we simply adjusted the output layers of the resnet model and added some new additions:

MaxPooling layer: This layer was selected over averagepooling2D due to max pooling's speciality in highlighting specific components of the images regardless of location, whereas averagepooling is more "broad" and fails to pinpoint the sharper details.

Dropout: This was introduced to counteract the problem of overfitting, that is common with CNN's of ResNet's size. The value was set to the benchmark of 0.5, which tends to be sufficient for the majority of CNN's.


In [ ]:
improved_ResNet = ResNet50(input_shape=(224,224,3),weights='imagenet',include_top=False)


improved_ResNet_outerlayer = improved_ResNet.output
improved_ResNet_outerlayer = MaxPooling2D(pool_size= (2,2))(improved_ResNet_outerlayer)
improved_ResNet_outerlayer = Dropout(0.5)(improved_ResNet_outerlayer)
improved_ResNet_outerlayer = Flatten()(improved_ResNet_outerlayer)

classes = 3

predictor = Dense(classes,activation='softmax')(improved_ResNet_outerlayer)

enhanced_Resnet = Model(inputs=improved_ResNet.input,outputs = predictor)

enhanced_Resnet.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:

enhanced_Resnet.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

z = enhanced_Resnet.fit_generator(generator=training_set,epochs=5,verbose=1,validation_data=test_set2)

In [17]:
#converting to sequential model 
newModel1 = Sequential()

# copying all the layers from previous model 

for layer in VGG.layers[:-2]:
  newModel1.add(layer)



# adding convulational layers 
newModel1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
newModel1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
newModel1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

classes = 3

# adding pooling layer

newModel1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


newModel1_last_layer = Flatten()(baseModel1.output)
newModel1_prediction = Dense(classes, activation='softmax')(vgg_last_layer)
improved_VGG = Model(inputs = newModel1.input, outputs= newModel1_prediction)

#summary to see added layers
improved_VGG.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [18]:

improved_VGG.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

y = improved_VGG.fit_generator(generator=training_set,epochs=5,verbose=1,validation_data=test_set2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/5
161/161 [==============================] - ETA: 0s - loss: 0.6052 - accuracy: 0.7519 

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


161/161 [==============================] - 2510s 16s/step - loss: 0.6052 - accuracy: 0.7519 - val_loss: 0.4486 - val_accuracy: 0.8284
Epoch 2/5
161/161 [==============================] - 2511s 16s/step - loss: 0.3735 - accuracy: 0.8552 - val_loss: 0.3649 - val_accuracy: 0.8641
Epoch 3/5
161/161 [==============================] - 2509s 16s/step - loss: 0.3227 - accuracy: 0.8764 - val_loss: 0.3231 - val_accuracy: 0.8866
Epoch 4/5
161/161 [==============================] - 2513s 16s/step - loss: 0.2943 - accuracy: 0.8902 - val_loss: 0.3201 - val_accuracy: 0.8742
Epoch 5/5
161/161 [==============================] - 2528s 16s/step - loss: 0.2758 - accuracy: 0.8929 - val_loss: 0.2797 - val_accuracy: 0.9022


In [23]:
improved_VGG_Test = improved_VGG.evaluate(test_set2)


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


41/41 [==============================] - 497s 12s/step - loss: 0.2839 - accuracy: 0.8952


In [22]:
final_prediction =  improved_VGG.predict_generator(test_set2)
predicted_classes_final = np.argmax(final_prediction, axis = 1)

actual = test_set2.classes
class_labels = list(test_set2.class_indices.keys())

report = classification_report(actual, predicted_classes_final, target_names = class_labels)
print(report)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


              precision    recall  f1-score   support

     COVID19       0.96      0.83      0.89       116
      NORMAL       0.78      0.85      0.82       317
   PNEUMONIA       0.93      0.92      0.93       855

    accuracy                           0.89      1288
   macro avg       0.89      0.87      0.88      1288
weighted avg       0.90      0.89      0.90      1288

